In [243]:
%load_ext autoreload
%autoreload 2

import torch
from transformers import LlamaConfig, LlamaForCausalLM, LlamaTokenizerFast, AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, GenerationConfig
from datasets import load_dataset 
from torch.utils.data import DataLoader
from datasets import Dataset, IterableDataset
from tqdm import tqdm

import torch
import litgpt
# import litdata as ld
# from litgpt.pretrain import initialize_weights
import lightning as L
import os
from scipy.spatial import distance
from sklearn.metrics import ndcg_score
import numpy as np
from torch.nn.functional import  kl_div

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
model_name = "meta-llama/Llama-3.2-3B"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [35]:
model.config

LlamaConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "float32",
  "transformers_version": "4.51.2",
  "use_cache": true,
  "vocab_size": 128256
}

In [7]:
data_files = {"train": "train/*.parquet", "validation": "validation/*.parquet"}
dataset: IterableDataset = load_dataset("../slim_pajama_1m", data_files=data_files, streaming=True)
dataset = dataset.remove_columns(['input_ids', 'attention_mask'])

val_dataloader = DataLoader(
    dataset['validation'], batch_size=1
)

Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

In [11]:
b

{'text': ["Design, production and dispatch of DM appeal letter with personalised variable segments. Multiple sheets matched by mrd barcode. Supporting appeal poster placed in Marketing Impact's high profile building frontage for the duration of the campaign."],
 'meta': {'redpajama_set_name': ['RedPajamaC4']}}

In [ ]:
class JSD(torch.nn.Module):
    def __init__(self):
        super(JSD, self).__init__()
        self.kl = torch.nn.KLDivLoss(reduction='batchmean', log_target=True)

    def forward(self, p: torch.tensor, q: torch.tensor):
        p, q = p.view(-1, p.size(-1)), q.view(-1, q.size(-1))
        m = (0.5 * (p + q)).log()
        return 0.5 * (self.kl(m, p.log()) + self.kl(m, q.log()))

In [255]:
model.eval()
model.cuda()

gen_config = GenerationConfig(
output_hidden_states=True, output_logits=True, return_dict_in_generate=True
)
jsd = JSD()
kl = torch.nn.KLDivLoss(reduction='batchmean', log_target=True)

ndcg_scores = []
with torch.no_grad():
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Embedding):            
            embb = module
    emb_weight = embb.weight.clone().detach()

    js_divs, kl_divs = [], []
    maxx = 300
    for i, b in tqdm(enumerate(val_dataloader), total=maxx) :
        if i > maxx: break

        curr_text = b['text'][0]
        curr_text = curr_text[:min(len(curr_text), 512) // 2]
        inputs = tokenizer(curr_text, return_tensors="pt")
        outputs = model.generate(input_ids=inputs['input_ids'].cuda(), attention_mask=inputs['attention_mask'].cuda(), generation_config=gen_config, max_new_tokens=20)

        logits = torch.stack(outputs.logits).squeeze() # (new_toks, vocab_size)
        if logits.dim() == 1:
            logits = logits[None, :]

        # logits = torch.so
        # ftmax(logits, dim=-1)

        last_hidden = torch.stack([outputs.hidden_states[0][-1][0, -1]] + [h[-1][0, 0] for  h in outputs.hidden_states[1:]]) # (new_toks, hidden_size)
        last_hidden = last_hidden

        kkn_logits = torch.cdist(last_hidden, emb_weight, compute_mode='use_mm_for_euclid_dist') # (new_toks, vocab_size)
        # kkn_logits = 1.0 / (kkn_logits + 1e-8)
        # js_div = jsd(kkn_logits, logits).cpu().item()
        # kl_div = kl(kkn_logits, logits).cpu().item()
        # js_divs.append(js_div)
        # kl_divs.append(kl_div)
        ndcg_scores += [ndcg_score(torch.softmax(logits, -1).cpu().numpy(), torch.softmax(1 / (kkn_logits + 1e-6), -1).cpu().numpy())]

np.mean(ndcg_scores)

 23%|██▎       | 69/300 [01:32<05:14,  1.36s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
100%|██████████| 300/300 [06:40<00:00,  1.37s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
301it [06:41,  1.33s/it]                         


0.9988970992516916

In [ ]:
# 0.9988970992516916 = ndcg

In [254]:
logits[None, :].shape

torch.Size([1, 128256])

In [247]:
kkn_logits.shape

torch.Size([1, 128256])

In [329]:
import ipywidgets as widgets
from IPython.display import display

class ExpandableBlocks:
    def __init__(self, top_words, dropdown_words, min_width='80px'):
        # Устанавливаем значения по умолчанию, если не предоставлены
        self.top_words = top_words 
        self.dropdown_words = dropdown_words 
        self.min_width = min_width
        
        # Проверяем, что для каждого верхнего слова есть список выпадающих слов
        for i in range(len(self.top_words)):
            if i not in self.dropdown_words:
                self.dropdown_words[i] = []
        
        # Создаем виджеты-кнопки для верхней строки
        self.top_buttons = []
        for i, word in enumerate(self.top_words):
            # Единый стиль для всех блоков (убрали розовую заливку)
            button = widgets.Button(
                description=word, 
                tooltip=word,  # Подсказка при наведении для длинных текстов
                style={'button_color': '#eeeeee'}, 
                layout=widgets.Layout(
                    width=self.min_width, 
                    height='50px',
                    overflow='auto',  # Скрываем выходящий за границы текст
                    text_overflow='ellipsis',  # Добавляем многоточие
                    white_space='wrap'  # Запрещаем перенос строк
                )
            )
            self.top_buttons.append(button)
        
        # Контейнер для верхних кнопок
        self.top_container = widgets.HBox(
            self.top_buttons, 
            layout=widgets.Layout(justify_content='center', align_items='flex-start')
        )
        
        # Создаем контейнеры для выпадающих блоков (по одному для каждой кнопки)
        self.expanded_containers = []
        for _ in range(len(self.top_words)):
            container = widgets.VBox(
                [], 
                layout=widgets.Layout(width=self.min_width, align_items='center')
            )
            self.expanded_containers.append(container)
        
        # Контейнер для дополнительных блоков с соответствующим количеством колонок
        self.dropdown_container = widgets.HBox(
            self.expanded_containers,
            layout=widgets.Layout(justify_content='center', align_items='flex-start')
        )
        
        # Назначаем обработчики событий для кнопок
        for i, button in enumerate(self.top_buttons):
            button.on_click(lambda b, idx=i: self.show_expanded(idx))
        
        # Основной контейнер
        self.main_container = widgets.VBox(
            [self.top_container, self.dropdown_container],
            layout=widgets.Layout(align_items='center')
        )
        
        # Для отслеживания текущего активного блока
        self.active_index = None
        
    def show_expanded(self, index):
        """Показать выпадающие блоки для выбранного индекса."""
        # Очищаем все контейнеры
        for container in self.expanded_containers:container.children = ()
        
        # Если нажали на уже активный блок, просто скрываем его
        if self.active_index == index:
            self.active_index = None
            return
        
        # Обновляем активный индекс
        self.active_index = index
        
        # Получаем список слов для данного индекса
        texts = self.dropdown_words.get(index, [])
        
        # Создаем новые дополнительные блоки только для выбранной колонки
        expanded_buttons = []
        for text in texts:
            btn = widgets.Button(
                description=text,
                tooltip=text,  # Подсказка при наведении для длинных текстов
                style={'button_color': 'white'},
                layout=widgets.Layout(
                    width=self.min_width,
                    min_width=self.min_width,
                    height='20px',
                    margin='5px 0px',
                    overflow='auto',
                    # text_overflow='ellipsis',
                    # white_space='nowrap'
                )
            )
            expanded_buttons.append(btn)
        
        # Помещаем дополнительные блоки в соответствующий контейнер
        self.expanded_containers[index].children = expanded_buttons
    
    def display(self):
        """Отобразить виджет."""
        display(self.main_container)


# Пример с пользовательскими словами, включая длинные названия
custom_top_words = ["Животные", "Растения", "Минералы", "Стихии"]
custom_dropdown_words = {
    0: ["Кошка", "Собака", "Лошадь"],
    1: ["Дуб", "Береза", "Роза", "Тюльпан"],
    2: ["Кварц", "Алмаз", "Золото"],
    3: []
}

# Создаем экземпляр с пользовательскими настройками
app = ExpandableBlocks(
    top_words=custom_top_words, 
    dropdown_words=custom_dropdown_words,
    min_width='150px'  # Увеличиваем ширину для более длинных слов
)
app.display()


In [336]:
text = 'I have a pen and '
TOP_K = 2

model.eval()
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(input_ids=inputs['input_ids'].cuda(), attention_mask=inputs['attention_mask'].cuda(), generation_config=gen_config, max_new_tokens=1)

for name, module in model.named_modules():
    if isinstance(module, torch.nn.Embedding):            
        embb = module
emb_weight = embb.weight.clone().detach()


tok_text = tokenizer.tokenize(text)

hiddens = torch.stack(outputs.hidden_states[0])[:, 0, 1:, :] # (layers, len, hiddensize)

kkn_logits = torch.cdist(hiddens.reshape(-1, hiddens.shape[-1]), emb_weight, compute_mode='use_mm_for_euclid_dist') # (layers, Len, vocab_size)
kkn_logits = kkn_logits.reshape(hiddens.shape[0], hiddens.shape[1], -1)

down_words = {}
for i in range(kkn_logits.shape[1]):
    curr = kkn_logits[:, i, :]
    x = torch.sort(curr).indices[:, 0]
    down_words[i] = [tokenizer.decode(t) for t in x]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [343]:
kkn_logits.reshape(hiddens.shape[:-1] + (-1,))

tensor([[[ 1.2579,  1.2421,  1.2637,  ...,  1.2261,  1.2261,  1.2261],
         [ 1.1850,  1.1988,  1.2359,  ...,  1.1425,  1.1425,  1.1426],
         [ 1.1987,  1.2351,  1.3284,  ...,  1.4053,  1.4053,  1.4053],
         [ 1.5635,  1.5837,  1.5695,  ...,  1.3579,  1.3579,  1.3579],
         [ 1.1733,  1.2276,  1.3226,  ...,  1.4375,  1.4375,  1.4375],
         [ 1.2864,  1.3112,  1.4115,  ...,  1.6557,  1.6557,  1.6557]],

        [[ 2.3636,  2.3665,  2.3660,  ...,  2.3038,  2.3038,  2.3038],
         [ 2.4102,  2.4356,  2.4342,  ...,  2.3579,  2.3579,  2.3579],
         [ 2.0973,  2.1131,  2.1290,  ...,  2.0570,  2.0570,  2.0570],
         [ 2.5646,  2.5955,  2.5797,  ...,  2.4969,  2.4969,  2.4969],
         [ 1.7336,  1.7362,  1.7542,  ...,  1.6744,  1.6744,  1.6744],
         [ 1.7192,  1.7206,  1.7469,  ...,  1.7072,  1.7072,  1.7072]],

        [[ 3.0658,  3.0566,  3.0861,  ...,  3.0380,  3.0380,  3.0380],
         [ 3.1900,  3.2169,  3.2076,  ...,  3.1517,  3.1517,  3.1517],
  

In [337]:
kkn_logits

tensor([[[ 1.2579,  1.2421,  1.2637,  ...,  1.2261,  1.2261,  1.2261],
         [ 1.1850,  1.1988,  1.2359,  ...,  1.1425,  1.1425,  1.1426],
         [ 1.1987,  1.2351,  1.3284,  ...,  1.4053,  1.4053,  1.4053],
         [ 1.5635,  1.5837,  1.5695,  ...,  1.3579,  1.3579,  1.3579],
         [ 1.1733,  1.2276,  1.3226,  ...,  1.4375,  1.4375,  1.4375],
         [ 1.2864,  1.3112,  1.4115,  ...,  1.6557,  1.6557,  1.6557]],

        [[ 2.3636,  2.3665,  2.3660,  ...,  2.3038,  2.3038,  2.3038],
         [ 2.4102,  2.4356,  2.4342,  ...,  2.3579,  2.3579,  2.3579],
         [ 2.0973,  2.1131,  2.1290,  ...,  2.0570,  2.0570,  2.0570],
         [ 2.5646,  2.5955,  2.5797,  ...,  2.4969,  2.4969,  2.4969],
         [ 1.7336,  1.7362,  1.7542,  ...,  1.6744,  1.6744,  1.6744],
         [ 1.7192,  1.7206,  1.7469,  ...,  1.7072,  1.7072,  1.7072]],

        [[ 3.0658,  3.0566,  3.0861,  ...,  3.0380,  3.0380,  3.0380],
         [ 3.1900,  3.2169,  3.2076,  ...,  3.1517,  3.1517,  3.1517],
  

In [334]:
tok_text

['I', 'Ġhave', 'Ġa', 'Ġpen', 'Ġand', 'Ġ']

In [335]:
app = ExpandableBlocks(
    top_words=tok_text, 
    dropdown_words=down_words,
    min_width='150px'  # Увеличиваем ширину для более длинных слов
)
app.display()


In [310]:
down_words

{0: ['Hello',
  'Hello',
  'Hello',
  'Hello',
  'Hello',
  'Hello',
  ' Hello',
  '.This',
  '.The',
  '.This',
  '.This',
  '.Please',
  '.We',
  'oo',
  '!!\n',
  'Hello',
  ' fellow',
  ' hello',
  ' hello',
  ' everyone',
  ' everyone',
  ' everyone',
  ' everyone',
  ' everyone',
  ' everyone',
  ' everyone',
  ' everyone',
  ',',
  ','],
 1: ['!',
  '!',
  '!',
  '!',
  '!',
  '!',
  '!',
  '!\n',
  '！',
  '!I',
  '.Please',
  '!I',
  '!I',
  '!!\n',
  '!!\n',
  'Welcome',
  ' hello',
  'Welcome',
  ' Welcome',
  ' Welcome',
  ' welcome',
  ' welcome',
  ' welcome',
  ' welcome',
  ' Welcome',
  ' Welcome',
  ' My',
  ' I',
  ' I']}

In [ ]:
inputs

{'input_ids': tensor([[128000,    791,  40825,    430,    279,   6246,    527,   7086,    369,
            527,    279,   1925,  33464,    315,  55507,   1624,    393,   8511,
           5165,   5657,     13,   1226,   2163,    279,    921,    321,  12052,
           8718,    773,    822,    520,    220,     18,   1428,    311,   1304,
            279,   1403,   6596,  35331,    311,    279,  40825,     13,    578,
          45688,    574,    264,  27946,  18916,    483,    369,    279,   1176,
           6596,    477,    779,    927,   1633,   2867,  25911,     13,   2360]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [301]:
torch.sort(curr).indices[:, :TOP_K]


tensor([[    0,  4999],
        [    0,  4999],
        [    0,  4999],
        [    0,  4999],
        [    0,  4999],
        [    0,  4999],
        [    0,  4999],
        [ 4999,     0],
        [ 6447,   998],
        [91597, 82927],
        [82927, 91597],
        [91597, 82927],
        [91597, 82927],
        [51447, 80395],
        [51447, 80395],
        [14262, 20776],
        [24748, 14262],
        [14262, 20776],
        [20776, 10788],
        [20776, 10788],
        [10788, 20776],
        [10788, 20776],
        [10788, 20776],
        [10788, 20776],
        [20776, 10788],
        [20776,  3092],
        [ 3092, 20776],
        [  358,  3092],
        [  358,  3092]], device='cuda:0')

In [297]:
kkn_logits.shape

torch.Size([29, 2, 128256])

In [308]:
tokenizer.decode(4999)

'!\n'

In [287]:
argsortkkn_logits.shape

torch.Size([29, 2, 128256])

In [280]:
kkn_logits.shape

torch.Size([58, 128256])

In [272]:
tok_text

['Hello', '!']

In [274]:
.shape

torch.Size([29, 2, 3072])